Importamos las librerias para el proyecto

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math


INGESTO EL ARCHIVO TRAIN.PARQUET Y COMIENZO A HACER EL EDA.

In [5]:
df_train = pd.read_parquet('C:/Users/Francisco/Desktop/Mi Proyecto 02/train.parquet')

df_train.head(2)

id                                                url  \
0  7048013474  https://boise.craigslist.org/apa/d/very-nice-b...   
1  7043931179  https://cosprings.craigslist.org/apa/d/colorad...   

             region                        region_url  price       type  \
0             boise      https://boise.craigslist.org   1350      house   
1  colorado springs  https://cosprings.craigslist.org   1115  apartment   

   sqfeet  beds  baths  cats_allowed  ...  wheelchair_access  \
0    1200     2    2.0             1  ...                  0   
1     694     1    1.0             1  ...                  0   

   electric_vehicle_charge  comes_furnished  laundry_options  parking_options  \
0                        0                0      w/d in unit  detached garage   
1                        0                0      w/d in unit          carport   

                                           image_url  \
0  https://images.craigslist.org/00B0B_cPiJMEheZe...   
1  https://images.craigslist.org/00R0R_5XAoSKvfrz...   

                                         description      lat     long  state  
0  Super cute row house in the Boise bench area. ...  43.5851 -116.225     id  
1  Windtree Apartment Homes currently has a spaci...  38.9137 -104.780     co  

[2 rows x 22 columns]


COPIO EL DF ORIGINAL A DF_TRAIN2 PARA PODER HACERLE CAMBIOS SIN PERDER LOS DATOS ORIGINALES.

In [7]:
#verificamos que la cantidad de filas sea la correcta
df_train2 = df_train
len(df_train2) 

346479

In [8]:
#Borramos los duplicados de la columna Description ya que asumimos que si la descipcion es identica se refiere a la misma vivienda y
# esto nos podria traer problemas en los modelos

df_train2 = df_train2.drop_duplicates('description')

len(df_train2)    #vemos la cantidad de registro con los que vamos a continuar el trabajo

254991

In [9]:
#verificamos la cantidad de nulos

print(df_train2.isnull().sum()) 

id                              0
url                             0
region                          0
region_url                      0
price                           0
type                            0
sqfeet                          0
beds                            0
baths                           0
cats_allowed                    0
dogs_allowed                    0
smoking_allowed                 0
wheelchair_access               0
electric_vehicle_charge         0
comes_furnished                 0
laundry_options             61200
parking_options            101866
image_url                       0
description                     1
lat                          1287
long                         1287
state                           0
dtype: int64


In [10]:
#completamos los valores null con empty

df_train2 = df_train2.fillna({'laundry_options': 'empty','parking_options': 'empty','description': 'empty' })

In [11]:
#vemos las columnas

df_train2.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state'],
      dtype='object')

In [12]:
#creamos la columna evaluatoria 

df_train2['category_binario'] = 0
df_train2['category_binario'] = np.where(df_train2['price']<=999 , 1 ,df_train2['category_binario'])
df_train2['category_binario'] = np.where(df_train2['price']> 999 , 0 ,df_train2['category_binario'])

In [13]:
#verficamos la cantidad de 0 y 1

df_train2['category_binario'].value_counts() 

0    149456
1    105535
Name: category_binario, dtype: int64


AHORA COMENZAMOS A REEMPLAZAR LAS VARIABLES CATEGORICAS POR 0 Y 1 (DUMMIES), ESTO NOS AYUDA A MEJORAR LA EXACTITUD DEL MODELO YA QUE PODEMOS USAR MAYOR 
CANTIDAD DE COLUMNAS PARA SU ENTRENAMIENTO.

In [14]:
df_train2.state.unique()    

array(['id', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms',
       'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl',
       'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi',
       'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy'], dtype=object)

In [15]:
#Creamos los dummis para State y borramos la columna 'state'

df_train2[['id', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms',
       'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl',
       'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi',
       'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy']] = pd.get_dummies(df_train2.state)
df_train2.drop(['state'],axis='columns', inplace=True)

In [16]:
df_train2.type.unique()

array(['house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'assisted living',
       'land'], dtype=object)

In [17]:
# Creamos los dummis para type y borramos la columna 'type'

df_train2[['house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'assisted living',
       'land']] = pd.get_dummies(df_train2.type)
df_train2.drop(['type'],axis='columns', inplace=True)

In [18]:
df_train.parking_options.unique()

array(['detached garage', 'carport', 'off-street parking', None,
       'attached garage', 'street parking', 'no parking', 'valet parking'],
      dtype=object)

In [19]:
# Creamos los dummis para parkimg options y borramos la columna 'parkimg_options'

df_train2[['detached garage', 'carport', 'off-street parking', 'unknown',
       'attached garage', 'street parking', 'no parking', 'valet parking']] = pd.get_dummies(df_train2.parking_options)
df_train2.drop(['parking_options'],axis='columns', inplace=True)

In [20]:
df_train2['category_binario'].dtype

dtype('int64')


BORRAMOS LAS COLUMNAS QUE NO SON RELEVANTES PARA NUESTRO MODELO.

In [21]:
df_train2= df_train2.drop(columns=['id', 'url', 'region_url', 'image_url','description', 'laundry_options','smoking_allowed'])

In [22]:
df_train2 = df_train2.drop(columns=['region'])

In [23]:
df_train2 = df_train2.drop(columns=['lat','long'])

ACA COMENZAMOS CON EL MODELO.
TESTEAMOS : ARBOL DE DESICION ,K-VECINOS, RANDOM FOREST, XGBOOST Y EL QUE MEJOR METRICA DIO FUE ARBOL DE DESICION

In [24]:
#IMPORTAMOS LIBRERIAS A UTILIZAR.

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split



In [25]:
# Crear el pipeline


pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier())
])

In [26]:
# Cargar los datos de entrada

X = df_train2.drop(columns=['price','category_binario'])
y = df_train2['category_binario']

In [27]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2)

In [28]:
# Ajustar el pipeline a los datos de entrenamiento

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', DecisionTreeClassifier())])

In [29]:
# Hacer predicciones en el conjunto de prueba

y_pred = pipe.predict(X_test)

In [30]:
# Evaluar la precisión del modelo

score = pipe.score(X_test, y_test)
print("Accuracy: ", score)

Accuracy:  0.8816643463597326



INGESTAMOS EL ARCHIVO TEST.PARQUET PARA REALIZAR PRUEBAS.

In [31]:
df_test = pd.read_parquet('C:/Users/Francisco/Desktop/Mi Proyecto 02/test.parquet')

CREAMOS LOS MISMOS DUMMIES QUE EN EL ARCHIVO TRAIN.

In [32]:
df_test[['id', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms',
       'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl',
       'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi',
       'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy']] = pd.get_dummies(df_test.state)
df_test.drop(['state'],axis='columns', inplace=True)

In [33]:
df_test.type.unique()

array(['manufactured', 'apartment', 'house', 'townhouse', 'condo',
       'duplex', 'cottage/cabin', 'loft', 'in-law', 'flat'], dtype=object)

In [34]:
df_test[['house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat']] = pd.get_dummies(df_test.type)
df_test.drop(['type'],axis='columns', inplace=True)

In [35]:
df_test = df_test.fillna({'parking_options' : 'vacio'})

In [36]:
df_test[['off-street parking', 'carport', 'attached garage', 'vacio',
       'street parking', 'detached garage', 'no parking', 'valet parking']] = pd.get_dummies(df_test.parking_options)
df_test.drop(['parking_options'],axis='columns', inplace=True)

In [37]:
df_test.vacio.unique()

array([0, 1], dtype=uint8)


ACA SE VERIFICA QUE LAS COLUMNAS DEL ARCHIVO TEST, NO SON LAS MISMAS QUE ACTUALMENTE ESTAN EN TRAIN2, POR LO CUAL SE PROCESE
A BORRAR COLUMNAS DE AMBOS DF PARA QUE SEAN IGUALES.

In [38]:
df_test.columns

Index(['id', 'url', 'region', 'region_url', 'sqfeet', 'beds', 'baths',
       'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'laundry_options',
       'image_url', 'description', 'lat', 'long', 'co', 'va', 'az', 'tn', 'tx',
       'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga',
       'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc',
       'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd',
       'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy', 'house',
       'apartment', 'duplex', 'loft', 'condo', 'manufactured', 'townhouse',
       'in-law', 'cottage/cabin', 'flat', 'off-street parking', 'carport',
       'attached garage', 'vacio', 'street parking', 'detached garage',
       'no parking', 'valet parking'],
      dtype='object')

In [39]:
#Creamos una copia del dataset nuevamente para no perder nada de lo trabajado anteriormente.

df_train1 = df_train2

In [40]:
df_train1.columns

Index(['price', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'category_binario', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa',
       'me', 'ms', 'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia',
       'fl', 'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh',
       'de', 'il', 'nm', 'mt', 'vt', 'wy', 'house', 'apartment', 'duplex',
       'loft', 'condo', 'manufactured', 'townhouse', 'in-law', 'cottage/cabin',
       'flat', 'assisted living', 'land', 'detached garage', 'carport',
       'off-street parking', 'unknown', 'attached garage', 'street parking',
       'no parking', 'valet parking'],
      dtype='object')

In [41]:
#Procedemos a borrar las sigueintes columnas del dataset de training

df_train1 = df_train1.drop(columns='price')
df_train1 = df_train1.drop(columns='assisted living')
df_train1 = df_train1.drop(columns='land')


In [44]:
#Procedemos a borrar las siguientes columnas del dataset de Testing
df_test = df_test.drop(columns=['region_url'])
df_test = df_test.drop(columns=['id'])
df_test = df_test.drop(columns=['url'])
df_test = df_test.drop(columns=['region'])
df_test = df_test.drop(columns=['smoking_allowed'])
df_test = df_test.drop(columns=['laundry_options'])
df_test = df_test.drop(columns=['image_url'])
df_test = df_test.drop(columns=['description'])
df_test = df_test.drop(columns=['lat', 'long'])

In [45]:
df_test['category_binario'] = 0

In [46]:
df_train1.columns

Index(['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'category_binario', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa',
       'me', 'ms', 'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia',
       'fl', 'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh',
       'de', 'il', 'nm', 'mt', 'vt', 'wy', 'house', 'apartment', 'duplex',
       'loft', 'condo', 'manufactured', 'townhouse', 'in-law', 'cottage/cabin',
       'flat', 'detached garage', 'carport', 'off-street parking', 'unknown',
       'attached garage', 'street parking', 'no parking', 'valet parking'],
      dtype='object')

In [47]:
df_test.columns

Index(['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'co',
       'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky',
       'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj',
       'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in',
       'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt',
       'wy', 'house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'off-street parking',
       'carport', 'attached garage', 'vacio', 'street parking',
       'detached garage', 'no parking', 'valet parking', 'category_binario'],
      dtype='object')


ORDENAMOS LAS COLUMNAS PARA QUE NOS QUEDEN IGUAL A TEST.

In [48]:
df_train1 = df_train1[['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'co',
       'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky',
       'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj',
       'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in',
       'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt',
       'wy', 'house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'off-street parking',
       'carport', 'attached garage', 'unknown', 'street parking',
       'detached garage', 'no parking', 'valet parking', 'category_binario']]

In [49]:
df_test = df_test.rename(columns={'vacio':'unknown'})

COMENZAMOS EL TESTEO EN EL ARCHIVO TEST.

In [50]:
# Cargar los datos de entrada

X = df_train1.drop(columns=['category_binario'])
y = df_train1['category_binario']

In [51]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2)

In [52]:
# Ajustar el pipeline a los datos de entrenamiento

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', DecisionTreeClassifier())])

In [53]:
# Hacer predicciones en el conjunto de prueba
y_pred = pipe.predict(X_test)


# Evaluar la precisión del modelo
score = pipe.score(X_test, y_test)
print(score)

0.8801545128335849


In [54]:
x1 =df_test.drop(columns=['category_binario'])

paraexportar = pipe.predict(x1)
score2 = pipe.score(X_test, y_test)
print(score2)

0.8801545128335849


In [ ]:
DF = pd.DataFrame(paraexportar)
DF.to_csv("eugeball.csv", header='pred', index=False)